# Fase 2: Transformación (Daily -> Monthly)

Este notebook ejecuta el pipeline de transformación para convertir datos diarios en series mensuales limpias.

**Objetivos:**
1. Limpiar valores centinela.
2. Normalizar categorías.
3. Agrupar por mes.
4. Imputar Gaps temporales.
5. Exportar reportes detallados a `outputs/reports/`.

In [1]:
# Celda 1: Importación de librerías y configuración de las rutas del proyecto
import pandas as pd
import os
import sys
from pathlib import Path

# Agregar src al path para importar el transformer
root_path = Path(os.getcwd()).parent
sys.path.append(str(root_path))

from src.preprocessing.transformer import DataTransformer

print(f"Directorio raíz: {root_path}")

Directorio raíz: c:\Users\USUARIO\Documents\Proyectos\Forecast_Pyme_Empanadas


## 1. Carga de Datos Raw

In [2]:
# Celda 2: Carga del archivo CSV con los datos transaccionales crudos (Raw)
raw_path = root_path / "data" / "01_raw" / "ventas_raw.csv"
df_raw = pd.read_csv(raw_path)
print(f"Dimensiones originales: {df_raw.shape}")
df_raw.head()

Dimensiones originales: (30099, 11)


,id,fecha,region,punto_venta,metodo_pago,tipo,unidades_vendidas,precio_venta,ingresos_totales,costo_unitario,created_at
0,1,2018-01-01,Manizales,Delivery,Efectivo,Carne,422,1800,759600,1300,2026-01-28T19:33:19.512953+00:00
1,2,2018-01-01,Manizales,Delivery,Efectivo,Pollo,293,1650,483450,1150,2026-01-28T19:33:19.682927+00:00
2,3,2018-01-01,Manizales,Delivery,Efectivo,Queso,156,1450,226200,1000,2026-01-28T19:33:19.845935+00:00
3,4,2018-01-01,Manizales,Delivery,Efectivo,Mixta,118,2000,236000,1400,2026-01-28T19:33:19.99027+00:00
4,5,2018-01-01,Manizales,Delivery,Efectivo,Chicharrón,95,1950,185250,1350,2026-01-28T19:33:20.147215+00:00


## 2. Ejecución del Transformer

In [3]:
# Celda 3: Ejecución del pipeline de transformación (Renombrar, Limpiar, Agrupar, Imputar)
transformer = DataTransformer(cutoff_date="2025-12-31")
df_monthly = transformer.run_pipeline(df_raw)

print(f"\nDimensiones finales mensuales: {df_monthly.shape}")
df_monthly.head()

2026-01-29 15:47:17,924 - INFO - Iniciando pipeline de transformación Fase 2 (Modo Estricto)...
2026-01-29 15:47:17,940 - INFO - Registros eliminados por fecha de corte (> 2025-12-31 00:00:00): 0
2026-01-29 15:47:17,942 - INFO - Valores centinela detectados y reemplazados: 0
2026-01-29 15:47:18,025 - INFO - Agregación mensual completada. Filas finales: 17380
2026-01-29 15:47:18,028 - INFO - Pipeline Fase 2 finalizado. Datos agregados pero sin completar gaps.



Dimensiones finales mensuales: (17380, 5)


,mes_fecha,ciudad,producto,canal,unidades
0,2018-01-01,BARRANQUILLA,ARROZ CON POLLO,CENTRO COMERCIAL,104.0
1,2018-01-01,BARRANQUILLA,ARROZ CON POLLO,DELIVERY,110.0
2,2018-01-01,BARRANQUILLA,CARNE,CENTRO COMERCIAL,606.0
3,2018-01-01,BARRANQUILLA,CARNE,DELIVERY,576.0
4,2018-01-01,BARRANQUILLA,CARNE DESMECHADA,CENTRO COMERCIAL,34.0


## 3. Verificación de Integridad

In [4]:
# Celda 4: Verificación de nulos sugerida y rango de fechas
print("Nulos finales:")
print(df_monthly.isnull().sum())

print("\nRango de fechas:")
print(f"Inicio: {df_monthly['mes_fecha'].min()}")
print(f"Fin: {df_monthly['mes_fecha'].max()}")

Nulos finales:
mes_fecha    0
ciudad       0
producto     0
canal        0
unidades     0
dtype: int64

Rango de fechas:
Inicio: 2018-01-01 00:00:00
Fin: 2025-12-01 00:00:00


## 4. Almacenamiento y Reportes Oficiales

In [5]:
# Celda 5: Almacenamiento .parquet y generación de reportes en outputs/reports/
output_dir_data = root_path / "data" / "02_cleansed"
output_dir_reports = root_path / "outputs" / "reports"

os.makedirs(output_dir_data, exist_ok=True)
os.makedirs(output_dir_reports, exist_ok=True)

output_path_parquet = output_dir_data / "ventas_mensuales.parquet"

try:
    # 1. Guardar archivo maestro Parquet
    df_monthly.to_parquet(output_path_parquet, engine='pyarrow', index=False)
    print(f"✅ Archivo Parquet guardado: {output_path_parquet}")
    
    # 2. Generar Reportes Fase 2 (JSON y MD)
    transformer.generate_report(df_raw, df_monthly, str(output_dir_reports))
    print(f"✅ Reportes oficiales generados en: {output_dir_reports}")
    print(f"   - p2_transformation_stats.json")
    print(f"   - p2_transformation_report.md")
except Exception as e:
    print(f"❌ Error en el proceso final: {e}")

✅ Archivo Parquet guardado: c:\Users\USUARIO\Documents\Proyectos\Forecast_Pyme_Empanadas\data\02_cleansed\ventas_mensuales.parquet

2026-01-29 15:47:18,089 - INFO - Reporte de integridad generado en: c:\Users\USUARIO\Documents\Proyectos\Forecast_Pyme_Empanadas\outputs\reports\p2_transformation_report.md



✅ Reportes oficiales generados en: c:\Users\USUARIO\Documents\Proyectos\Forecast_Pyme_Empanadas\outputs\reports
   - p2_transformation_stats.json
   - p2_transformation_report.md
